<a href="https://colab.research.google.com/github/caseyfengl/linkedin-learning-data-dump/blob/main/lil_content_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
#import json
import datetime

In [ ]:
# some useful functions

## convert ms to datetime
def to_datetime(ms):
  dt = datetime.datetime.utcfromtimestamp(ms / 1000)
  return dt
###to_datetime(1645603250298)

## convert datetime to ms
def to_ms(dt):
  ms = dt.timestamp()*1000
  return int(ms)
###to_ms(datetime.datetime.now())

In [ ]:
# generate an access token
endpoint = "https://www.linkedin.com/oauth/v2/accessToken?grant_type=client_credentials&client_id=86trc3hjuhjua2&client_secret=Pbho7TZwNtAPZhc8"
response = requests.post(endpoint)
while response.status_code != 200:
  response = requests.post(endpoint)
token = response.json()['access_token']

Learner Summary for past 12 months

In [ ]:
# initiate some time variables
startedAt = datetime.datetime.now()-datetime.timedelta(days=14)
aYearAgo = datetime.datetime.now()-datetime.timedelta(days=365)

# set up some API params
headers = {"Authorization":"Bearer "+token}

# some other variables for the loop
learner_summary = pd.DataFrame()
keys_to_extract = {'email','enterpriseGroups','name','uniqueUserId'}

# Loop through 26 fortnights
while startedAt>aYearAgo:
  endpoint = "https://api.linkedin.com/v2/learningActivityReports?q=criteria&start=1&count=75000&startedAt={}&"\
              "timeOffset.duration=2&aggregationCriteria.primary=INDIVIDUAL&timeOffset.unit=WEEK&sortBy.engagementMetricType=LOGINS&"\
              "contentSource=LINKEDIN_LEARNING".format(to_ms(startedAt))
  response = requests.get(endpoint,headers=headers)
  # make sure status = 200, retry if not
  while response.status_code != 200:
    response = requests.get(endpoint,headers=headers)
  # produce report for the loop fortnight
  ls_df = pd.DataFrame(response.json()['elements'])
  loop_report = pd.DataFrame()
  # loop through the employees
  for i in range(0,len(ls_df)):
    # parse learnerDetails
    extracted_dict = {key: ls_df['learnerDetails'][i][key] for key in ls_df['learnerDetails'][i].keys() & keys_to_extract}
    broader_attributes = pd.DataFrame.from_dict(extracted_dict, orient = "index").T
    custom_attributes = pd.DataFrame.from_dict(ls_df['learnerDetails'][i]['customAttributes'], orient = "index").T
    learner_details = pd.concat([broader_attributes,custom_attributes], axis=1)
    # parse activities
    activities = pd.DataFrame(ls_df['activities'][i]).fillna('')
    activities = activities.set_index(activities['assetType']+' '+activities['engagementType'])
    activities_details = activities[['engagementValue']].T.reset_index(drop=True)
    # concat information of the employee
    temp = pd.concat([learner_details,activities_details],axis=1)
    loop_report = loop_report.append(temp)
  # append fortnightly report to learner summary report
  learner_summary = learner_summary.append(loop_report)
  # go back 14 days for the next loop
  startedAt -= datetime.timedelta(days=14)

# Append data from the last day of the year (365 days)
endpoint = "https://api.linkedin.com/v2/learningActivityReports?q=criteria&start=1&count=75000&startedAt={}&"\
            "timeOffset.duration=1&aggregationCriteria.primary=INDIVIDUAL&timeOffset.unit=DAY&sortBy.engagementMetricType=LOGINS&"\
            "contentSource=LINKEDIN_LEARNING".format(to_ms(datetime.datetime.now()-datetime.timedelta(days=365)))
response = requests.get(endpoint,headers=headers)
# make sure status = 200, retry if not
while response.status_code != 200:
  response = requests.get(endpoint,headers=headers)
# produce report for the loop fortnight
ls_df = pd.DataFrame(response.json()['elements'])
loop_report = pd.DataFrame()
# loop through the employees
for i in range(0,len(ls_df)):
  # parse learnerDetails
  extracted_dict = {key: ls_df['learnerDetails'][i][key] for key in ls_df['learnerDetails'][i].keys() & keys_to_extract}
  broader_attributes = pd.DataFrame.from_dict(extracted_dict, orient = "index").T
  custom_attributes = pd.DataFrame.from_dict(ls_df['learnerDetails'][i]['customAttributes'], orient = "index").T
  learner_details = pd.concat([broader_attributes,custom_attributes], axis=1)
  # parse activities
  activities = pd.DataFrame(ls_df['activities'][i]).fillna('')
  activities = activities.set_index(activities['assetType']+' '+activities['engagementType'])
  activities_details = activities[['engagementValue']].T.reset_index(drop=True)
  # concat information of the employee
  temp = pd.concat([learner_details,activities_details],axis=1)
  loop_report = loop_report.append(temp)
# append fortnightly report to learner summary report
learner_summary = learner_summary.append(loop_report)

# group data by employees and sum up the numbers
learner_summary[[' DAYS_ACTIVE',
       'VIDEO SECONDS_VIEWED', 'ARTICLE VIEWS', 'ARTICLE COMPLETIONS',
       'AUDIO VIEWS', 'AUDIO COMPLETIONS', 'BOOK VIEWS', 'BOOK COMPLETIONS',
       'LEARNING_COLLECTION VIEWS', 'LEARNING_COLLECTION COMPLETIONS',
       'COURSE VIEWS', 'COURSE COMPLETIONS', 'DOCUMENT VIEWS',
       'DOCUMENT COMPLETIONS', 'EVENT VIEWS', 'EVENT COMPLETIONS',
       'LEARNING_PATH VIEWS', 'LEARNING_PATH COMPLETIONS', 'VIDEO VIEWS',
       'VIDEO COMPLETIONS']] = \
learner_summary.groupby(['uniqueUserId'])[[' DAYS_ACTIVE',
       'VIDEO SECONDS_VIEWED', 'ARTICLE VIEWS', 'ARTICLE COMPLETIONS',
       'AUDIO VIEWS', 'AUDIO COMPLETIONS', 'BOOK VIEWS', 'BOOK COMPLETIONS',
       'LEARNING_COLLECTION VIEWS', 'LEARNING_COLLECTION COMPLETIONS',
       'COURSE VIEWS', 'COURSE COMPLETIONS', 'DOCUMENT VIEWS',
       'DOCUMENT COMPLETIONS', 'EVENT VIEWS', 'EVENT COMPLETIONS',
       'LEARNING_PATH VIEWS', 'LEARNING_PATH COMPLETIONS', 'VIDEO VIEWS',
       'VIDEO COMPLETIONS']].transform('sum')
learner_summary = learner_summary.drop_duplicates(subset=['uniqueUserId']).reset_index(drop=True)

# drop lists in 'enterpriseGroups' column
groups = []
for each_list in learner_summary['enterpriseGroups']:
  if each_list == []:
    groups.append('')
  elif each_list is None:
    groups.append('')
  else:
    group = ', '.join(str(e) for e in each_list)
    groups.append(group)
# overwrite the original 'enterpriseGroups' column
learner_summary['enterpriseGroups'] = groups

# convert seconds to hours
hours = []
for second in learner_summary['VIDEO SECONDS_VIEWED']:
  hour = round(second/3600, 4)
  hours.append(hour)
learner_summary['VIDEO SECONDS_VIEWED'] = hours

# rename columns
learner_summary = \
learner_summary.rename(columns={'uniqueUserId':'UniqueUserID','name':'Name','email':'Email','enterpriseGroups':'Groups','Employee ID':'EmployeeID',' DAYS_ACTIVE':'DaysLoggedIn',
                                'VIDEO SECONDS_VIEWED':'HoursViewed','ARTICLE VIEWS':'ArticleViews','ARTICLE COMPLETIONS':'ArticleCompletions','AUDIO VIEWS':'AudioViews',
                                'AUDIO COMPLETIONS':'AudioCompletions','BOOK VIEWS':'BookViews','BOOK COMPLETIONS':'BookCompletions','LEARNING_COLLECTION VIEWS':'CollectionViews',
                                'LEARNING_COLLECTION COMPLETIONS':'CollectionCompletions','COURSE VIEWS':'CourseViews','COURSE COMPLETIONS':'CourseCompletions','DOCUMENT VIEWS':'DocumentViews',
                                'DOCUMENT COMPLETIONS':'DocumentCompletions','EVENT VIEWS':'EventViews','EVENT COMPLETIONS':'EventCompletions','LEARNING_PATH VIEWS':'LearningPathViews',
                                'LEARNING_PATH COMPLETIONS':'LearningPathCompletions','VIDEO VIEWS':'VideoViews','VIDEO COMPLETIONS':'VideoCompletions'})

In [ ]:
learner_summary.columns

Index(['uniqueUserId', 'name', 'email', 'enterpriseGroups', 'Role',
       'Employee ID', 'Company', 'homeorganisationunit', ' DAYS_ACTIVE',
       'VIDEO SECONDS_VIEWED', 'ARTICLE VIEWS', 'ARTICLE COMPLETIONS',
       'AUDIO VIEWS', 'AUDIO COMPLETIONS', 'BOOK VIEWS', 'BOOK COMPLETIONS',
       'LEARNING_COLLECTION VIEWS', 'LEARNING_COLLECTION COMPLETIONS',
       'COURSE VIEWS', 'COURSE COMPLETIONS', 'DOCUMENT VIEWS',
       'DOCUMENT COMPLETIONS', 'EVENT VIEWS', 'EVENT COMPLETIONS',
       'LEARNING_PATH VIEWS', 'LEARNING_PATH COMPLETIONS', 'VIDEO VIEWS',
       'VIDEO COMPLETIONS', 'Function', 'State', 'Team'],
      dtype='object')

In [ ]:
learner_summary.tail()

,Name,Groups,Email,UniqueUserID,Role,EmployeeID,Company,homeorganisationunit,DaysLoggedIn,HoursViewed,...,DocumentCompletions,EventViews,EventCompletions,LearningPathViews,LearningPathCompletions,VideoViews,VideoCompletions,Function,State,Team
1555,Fiona Price,"Aust Post, Delivery Digital & Data",Fiona.Price2@auspost.com.au,Fiona.Price2@auspost.com.au,Graduate Agile Business Analyst,11075153,Aust Post,Delivery Digital & Data,1,0.4183,...,0,0,0,0,0,8,8,NaN,NaN,NaN
1556,Roy Valerio,"Aust Post, Productivity Platforms",roy.valerio@auspost.com.au,roy.valerio@auspost.com.au,Technical Specialist - Productivity Platforms,40518373,Aust Post,Productivity Platforms,1,0.0000,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1557,Ashlinn Rodden,Business_Sales,ASHLINN.RODDEN@AUSPOST.COM.AU,ASHLINN.RODDEN@AUSPOST.COM.AU,NaN,NaN,NaN,NaN,1,0.0081,...,0,0,0,0,0,1,0,NaN,NaN,NaN
1558,Marie Ellen,"Aust Post, International Pricing",Marie.Ellen@auspost.com.au,Marie.Ellen@auspost.com.au,Head of International Pricing,11053804,Aust Post,International Pricing,1,0.0725,...,0,0,0,0,0,2,2,NaN,NaN,NaN
1559,Trevor Baker,"Decipha, Decipha State Operations - SA (D)",Trevor.Baker@apgteam.com.au,Trevor.Baker@apgteam.com.au,NaN,11047999,Decipha,Decipha State Operations - SA (D),1,0.0000,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [ ]:
pd.__version__

'1.3.5'

In [ ]:
!python -V

Python 3.7.12
